In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [4]:
#restore HW2 model's tensor by name

tf.reset_default_graph()
restore_saver = tf.train.import_meta_graph( "./model/Team11_HW2.ckpt.meta")
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

#get the softmax layer
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")

#define optimizer and loss function
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()


HW3.1 soft max only

In [5]:
import time

n_epochs = 1000
early_stop_trigger_step = 20
early_stop_cnt = 0
batch_size = 30

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./model/Team11_HW2.ckpt")
    
    
    for var in output_layer_vars:
        var.initializer.run()
        
    t0 = time.time()
    
    best_loss = np.infty
    for epoch in range(n_epochs):
        
        #random select training data
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        
        if loss_val < best_loss:
            best_loss = loss_val
            early_stop_cnt = 0
            save_path = five_frozen_saver.save(sess, "./Team11_HW3_1.ckpt")
        else:
            early_stop_cnt += 1
            if early_stop_cnt > early_stop_trigger_step:
                print("Early stop!")
                break
        print("{}\tValid loss: {:.4f}\tBest loss: {:.4f}\tACC: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Training time: {:.1f}sec".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./Team11_HW3_1.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("test acc: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./model/Team11_HW2.ckpt
0	Valid loss: 15.6580	Best loss: 15.6580	ACC: 8.00%
1	Valid loss: 12.6536	Best loss: 12.6536	ACC: 8.00%
2	Valid loss: 8.8642	Best loss: 8.8642	ACC: 9.33%
3	Valid loss: 6.3263	Best loss: 6.3263	ACC: 10.67%
4	Valid loss: 3.7347	Best loss: 3.7347	ACC: 11.33%
5	Valid loss: 2.2184	Best loss: 2.2184	ACC: 32.00%
6	Valid loss: 1.4932	Best loss: 1.4932	ACC: 43.33%
7	Valid loss: 1.3058	Best loss: 1.3058	ACC: 50.00%
8	Valid loss: 1.4664	Best loss: 1.3058	ACC: 44.00%
9	Valid loss: 1.3992	Best loss: 1.3058	ACC: 49.33%
10	Valid loss: 1.3318	Best loss: 1.3058	ACC: 49.33%
11	Valid loss: 1.4148	Best loss: 1.3058	ACC: 44.67%
12	Valid loss: 1.3337	Best loss: 1.3058	ACC: 46.00%
13	Valid loss: 1.3658	Best loss: 1.3058	ACC: 40.00%
14	Valid loss: 1.2451	Best loss: 1.2451	ACC: 50.00%
15	Valid loss: 1.2783	Best loss: 1.2451	ACC: 44.67%
16	Valid loss: 1.3349	Best loss: 1.2451	ACC: 45.33%
17	Valid loss: 1.2985	Best loss: 1.2451	ACC: 54.67%
18	Vali

HW3.2 cache 5th layer

In [6]:
hidden5_out = tf.get_default_graph().get_tensor_by_name("hidden5_out:0")

n_epochs = 1000
batch_size = 30

early_stop_trigger_step = 20
early_stop_cnt = 0

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./model/Team11_HW2.ckpt")
    for var in output_layer_vars:
        var.initializer.run()

    t0 = time.time()
    #X_train2(500,784)
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
    best_loss = np.infty
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))  #create 1~500 rnd
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./Team11_HW3_2.ckpt")
            best_loss = loss_val
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1
            if early_stop_cnt > early_stop_trigger_step:
                print("Early stopping!")
                break
        print("{}\tValid loss: {:.4f}\tBest loss: {:.4f}\tACC: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Training time: {:.1f}sec".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./Team11_HW3_2.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("test acc: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./model/Team11_HW2.ckpt
0	Valid loss: 15.7288	Best loss: 15.7288	ACC: 8.00%
1	Valid loss: 12.2579	Best loss: 12.2579	ACC: 10.00%
2	Valid loss: 9.2425	Best loss: 9.2425	ACC: 10.67%
3	Valid loss: 6.5090	Best loss: 6.5090	ACC: 11.33%
4	Valid loss: 4.1569	Best loss: 4.1569	ACC: 12.00%
5	Valid loss: 2.6744	Best loss: 2.6744	ACC: 24.00%
6	Valid loss: 1.7984	Best loss: 1.7984	ACC: 34.67%
7	Valid loss: 1.6328	Best loss: 1.6328	ACC: 38.00%
8	Valid loss: 1.5937	Best loss: 1.5937	ACC: 38.00%
9	Valid loss: 1.5425	Best loss: 1.5425	ACC: 43.33%
10	Valid loss: 1.5245	Best loss: 1.5245	ACC: 40.67%
11	Valid loss: 1.5157	Best loss: 1.5157	ACC: 42.00%
12	Valid loss: 1.4923	Best loss: 1.4923	ACC: 42.67%
13	Valid loss: 1.4762	Best loss: 1.4762	ACC: 43.33%
14	Valid loss: 1.4598	Best loss: 1.4598	ACC: 44.00%
15	Valid loss: 1.4548	Best loss: 1.4548	ACC: 42.00%
16	Valid loss: 1.4288	Best loss: 1.4288	ACC: 46.67%
17	Valid loss: 1.4334	Best loss: 1.4288	ACC: 43.33%
18	Va

HW3.3 Load HW2 model again, now use 4th hidden layer's output and feed them into new softmax layer:

In [7]:
reset_graph()

n_outputs = 5

restore_saver = tf.train.import_meta_graph("./model/Team11_HW2.ckpt.meta")
he_init = tf.contrib.layers.variance_scaling_initializer()
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden4_out = tf.get_default_graph().get_tensor_by_name("hidden4_out:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")


#define optimizer and loss function
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [8]:
n_epochs = 1000
batch_size = 30


early_stop_trigger_step = 20
early_stop_cnt = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./model/Team11_HW2.ckpt")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./Team11_HW3_3.ckpt")
            best_loss = loss_val
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1
            if early_stop_cnt > early_stop_trigger_step:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./Team11_HW3_3.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./model/Team11_HW2.ckpt
0	Validation loss: 3.133135	Best loss: 3.133135	Accuracy: 44.00%
1	Validation loss: 2.350898	Best loss: 2.350898	Accuracy: 44.00%
2	Validation loss: 2.064627	Best loss: 2.064627	Accuracy: 34.00%
3	Validation loss: 2.552538	Best loss: 2.064627	Accuracy: 47.33%
4	Validation loss: 2.335384	Best loss: 2.064627	Accuracy: 30.00%
5	Validation loss: 2.001016	Best loss: 2.001016	Accuracy: 44.00%
6	Validation loss: 1.633297	Best loss: 1.633297	Accuracy: 42.67%
7	Validation loss: 1.681264	Best loss: 1.633297	Accuracy: 49.33%
8	Validation loss: 1.913101	Best loss: 1.633297	Accuracy: 48.67%
9	Validation loss: 2.223051	Best loss: 1.633297	Accuracy: 48.00%
10	Validation loss: 1.949555	Best loss: 1.633297	Accuracy: 44.67%
11	Validation loss: 1.692915	Best loss: 1.633297	Accuracy: 43.33%
12	Validation loss: 1.848531	Best loss: 1.633297	Accuracy: 38.67%
13	Validation loss: 1.807144	Best loss: 1.633297	Accuracy: 48.67%
14	Validation loss: 

HW3.4  Unfreeze top 2 layers

In [9]:
learning_rate = 0.01
unfrozen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam3")
training_op = optimizer.minimize(loss, var_list=unfrozen_vars)

init = tf.global_variables_initializer()
two_frozen_saver = tf.train.Saver()

In [11]:
n_epochs = 1000
batch_size = 30


early_stop_trigger_step = 20
early_stop_cnt = 0


with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "./Team11_HW3_3.ckpt")
    best_loss = np.infty
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = two_frozen_saver.save(sess, "./Team11_HW3_4.ckpt")
            best_loss = loss_val
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1
            if early_stop_cnt > early_stop_trigger_step:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    two_frozen_saver.restore(sess, "./Team11_HW3_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./Team11_HW3_3.ckpt
0	Validation loss: 1.960130	Best loss: 1.960130	Accuracy: 46.67%
1	Validation loss: 1.950920	Best loss: 1.950920	Accuracy: 35.33%
2	Validation loss: 1.490337	Best loss: 1.490337	Accuracy: 52.00%
3	Validation loss: 1.371305	Best loss: 1.371305	Accuracy: 48.00%
4	Validation loss: 1.259536	Best loss: 1.259536	Accuracy: 54.67%
5	Validation loss: 1.252204	Best loss: 1.252204	Accuracy: 48.00%
6	Validation loss: 1.262321	Best loss: 1.252204	Accuracy: 54.00%
7	Validation loss: 1.258616	Best loss: 1.252204	Accuracy: 50.67%
8	Validation loss: 1.271679	Best loss: 1.252204	Accuracy: 48.00%
9	Validation loss: 1.203111	Best loss: 1.203111	Accuracy: 56.00%
10	Validation loss: 1.162395	Best loss: 1.162395	Accuracy: 49.33%
11	Validation loss: 1.004879	Best loss: 1.004879	Accuracy: 63.33%
12	Validation loss: 1.206293	Best loss: 1.004879	Accuracy: 55.33%
13	Validation loss: 1.238659	Best loss: 1.004879	Accuracy: 53.33%
14	Validation loss: 1.11